In [1]:
import hail as hl
from hail.experimental import ManhattanPreprocessor
from hail.experimental import TileGenerator

In [2]:
hl.init()

Running on Apache Spark version 2.2.0
SparkUI available at http://10.1.7.107:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version devel-013c5378964e
NOTE: This is a beta version. Interfaces may change
  during the beta period. We recommend pulling
  the latest changes weekly.


In [3]:
colors = {}
for i in range(1,23):
    if i%2==0:
        colors[str(i)] = '#0e6d19'
    else:
        colors[str(i)] = "#000000"
colors['X'] = "#000000"

In [4]:
# add gene labels
mt = hl.read_matrix_table('/Users/maccum/manhattan_data/raw/caffeine_mt')
intervals = hl.import_locus_intervals('/Users/maccum/Downloads/genes.37.interval_list')
mt = mt.key_rows_by('locus')
mt = mt.annotate_rows(gene = intervals[mt.locus].target).key_rows_by('locus','alleles')

2018-08-29 17:53:53 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (user-specified)
  Loading column 'f1' as type 'int32' (user-specified)
  Loading column 'f2' as type 'int32' (user-specified)
  Loading column 'f3' as type 'str' (user-specified)
  Loading column 'f4' as type 'str' (user-specified)



In [5]:
# add manhattan plotting data to mt
#mt = hl.read_matrix_table('/Users/maccum/manhattan_data/raw/caffeine_mt')
mp = ManhattanPreprocessor(mt.locus, mt.phenotype, mt.p_value)
manhat_mt = mp.add_manhattan_data(colors)
manhat_mt.write('/Users/maccum/manhattan_data/with_plot_fields/caffeine_mt2', overwrite=True)

2018-08-29 17:53:54 Hail: INFO: Coerced sorted dataset
2018-08-29 17:53:55 Hail: INFO: Coerced sorted dataset
2018-08-29 17:53:56 Hail: INFO: Coerced sorted dataset
2018-08-29 17:53:56 Hail: INFO: Ordering unsorted dataset with network shuffle
2018-08-29 17:53:58 Hail: INFO: Coerced sorted dataset
2018-08-29 17:54:00 Hail: INFO: wrote 10879 items in 16 partitions to /Users/maccum/manhattan_data/with_plot_fields/caffeine_mt2


In [6]:
# read manhattan plotting data from disk and start tile generation
manhat_mt = hl.read_matrix_table('/Users/maccum/manhattan_data/with_plot_fields/caffeine_mt2')
tg = TileGenerator(manhat_mt,
                  dest='/Users/maccum/manhattan_data/plots/caffeine_plots_2',
                  regen=True,
                  x_margin=10000000,
                  y_margin=2)

tg.generate_tile_layer(phenotype='caffeine_consumption', zoom=2, new_log_file=True)


Zoom level: 2 |██████████████████████████████████████████████████| 100.0% Complete


In [15]:
manhat_mt.gp_range.show()


+--------------+--------------+
| gp_range.min | gp_range.max |
+--------------+--------------+
|        int32 |        int64 |
+--------------+--------------+
|       904164 |   3035120653 |
+--------------+--------------+



In [7]:
tg.generate_tile_layer(phenotype='caffeine_consumption',zoom=3)

Zoom level: 3 |██████████████████████████████████████████████████| 100.0% Complete


In [8]:
tg.generate_tile_layer(phenotype='caffeine_consumption',zoom=3)
tg.generate_tile_layer(phenotype='caffeine_consumption',zoom=4)
tg.generate_tile_layer(phenotype='caffeine_consumption',zoom=5)
tg.generate_tile_layer(phenotype='caffeine_consumption',zoom=6)
tg.generate_tile_layer(phenotype='caffeine_consumption',zoom=7)
tg.generate_tile_layer(phenotype='caffeine_consumption',zoom=8)

Zoom level: 3 |██████████████████████████████████████████████████| 100.0% Complete
Zoom level: 4 |██████████████████████████████████████████████████| 100.0% Complete
Zoom level: 5 |██████████████████████████████████████████████████| 100.0% Complete
Zoom level: 6 |██████████████████████████████████████████████████| 100.0% Complete
Zoom level: 7 |██████████████████████████████████████████████████| 100.0% Complete
Zoom level: 8 |██████████████████████████████████████████████████| 100.0% Complete


In [8]:
manhat_mt.filter_entries(manhat_mt.neg_log_pval > 8).entries().show()

+---------------+------------+-----------------+---------+----------------------+
| locus         | alleles    | global_position | color   | phenotype            |
+---------------+------------+-----------------+---------+----------------------+
| locus<GRCh37> | array<str> |           int64 | str     | str                  |
+---------------+------------+-----------------+---------+----------------------+
| 3:11677077    | ["C","T"]  |       504127070 | #000000 | caffeine_consumption |
| 3:52099441    | ["T","C"]  |       544549434 | #000000 | caffeine_consumption |
| 19:47224607   | ["C","T"]  |      2706668928 | #000000 | caffeine_consumption |
+---------------+------------+-----------------+---------+----------------------+

+-------------+-------------+-------------+--------------+
|     min_nlp |     max_nlp |     p_value | neg_log_pval |
+-------------+-------------+-------------+--------------+
|     float64 |     float64 |     float64 |      float64 |
+-------------+----------

In [84]:
#ht = hl.utils.range_table(5)
#ht = ht.annotate(y = ht.idx%2, e = "#00000", f = hl.str(ht.idx%2==0), alleles=hl.array(['C','T']))
#ds = ht.aggregate(hl.agg.downsample(ht.idx, ht.y, [ht['e'], ht['f'], ht['alleles']]))
#ds
#hl.str(hl.parse_locus('1:100')).value
#hl.str(hl.bool(True)).value
#hl.str(hl.array(['C','T'])).value
label = ['foo', 3]
point = {'gp':1000, 'nlp': 10}
point['label'] = label
point

{'gp': 1000, 'label': ['foo', 3], 'nlp': 10}

In [67]:
str(hl.array(['C','T']).value)

"['C', 'T']"

In [16]:
metadata = [
    {
    'gp': 504127070,
    'nlp': 8.19918,
    'chrPos': '3:11677077',
    'alleles': '[C,T]', 
    'p': 2.74879
    },
    {
    'gp': 544549434,
    'nlp': 9.76749,
    'chrPos': "3:52099441",
    'alleles': '[T,C]', 
    'p': 5.72837
    },
]
import json
with open('test.json', 'w') as outfile:
    json.dump(metadata, outfile)
    
    
with open('test.json') as data_file:    
    read_metadata = json.load(data_file)
read_metadata

[{'alleles': '[C,T]',
  'chrPos': '3:11677077',
  'gp': 504127070,
  'nlp': 8.19918,
  'p': 2.74879},
 {'alleles': '[T,C]',
  'chrPos': '3:52099441',
  'gp': 544549434,
  'nlp': 9.76749,
  'p': 5.72837}]

In [17]:
manhat_mt.describe()

----------------------------------------
Global fields:
    'gp_range': struct {
        min: int32, 
        max: int64
    } 
----------------------------------------
Column fields:
    'phenotype': str 
    'min_nlp': float64 
    'max_nlp': float64 
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'global_position': int64 
    'color': str 
----------------------------------------
Entry fields:
    'p_value': float64 
    'neg_log_pval': float64 
----------------------------------------
Column key: ['phenotype']
Row key: ['locus', 'alleles']
Partition key: ['locus', 'alleles']
----------------------------------------


2018-08-28 17:28:11 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (user-specified)
  Loading column 'f1' as type 'int32' (user-specified)
  Loading column 'f2' as type 'int32' (user-specified)
  Loading column 'f3' as type 'str' (user-specified)
  Loading column 'f4' as type 'str' (user-specified)



In [17]:
arr = hl.array(['locus', 'alleles']).extend(hl.array(['foo', 'bar']))

In [19]:
arr[2:].value

['foo', 'bar']

In [20]:
['a', 'b', 'c', 'd'][2:]

['c', 'd']

In [9]:
(2047/2048)*8

7.99609375